In [1]:
'layer.py'
import torch
import torch.nn.functional as F
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, dropout=0., act=F.relu):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.act = act
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight)

    def forward(self, input, adj):
        input = F.dropout(input, self.dropout, self.training)
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        output = self.act(output)
        return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [2]:
'model.py'

import torch.nn as nn


class GCNModelAE(nn.Module):
    def __init__(self, input_feat_dim, hidden_dim1, hidden_dim2, dropout):
        super(GCNModelAE, self).__init__()
        self.gc1 = GraphConvolution(input_feat_dim, hidden_dim1, dropout, act=F.relu)
        self.gc2 = GraphConvolution(hidden_dim1, hidden_dim2, dropout, act=lambda x: x)
        self.dc = InnerProductDecoder(dropout, act=lambda x: x)

    def encode(self, x, adj):
        hidden1 = self.gc1(x, adj)
        return self.gc2(hidden1, adj)

    def forward(self, x, adj):
        embeddings = self.encode(x, adj)
        return self.dc(embeddings), embeddings


class InnerProductDecoder(nn.Module):
    """Decoder for using inner product for prediction."""

    def __init__(self, dropout, act=torch.sigmoid):
        super(InnerProductDecoder, self).__init__()
        self.dropout = dropout
        self.act = act

    def forward(self, z):
        z = F.dropout(z, self.dropout, training=self.training)
        adj = self.act(torch.mm(z, z.t()))
        return adj

In [3]:
"optimiser.py"

import torch.nn.modules.loss

def loss_function(preds, labels, norm, pos_weight):
    cost = norm * F.binary_cross_entropy_with_logits(preds, labels, pos_weight=pos_weight)
    return cost

In [4]:
import pickle

import networkx as nx
import numpy as np
import scipy.sparse as sp

In [5]:
def preprocess_graph(adj):
    adj = sp.coo_matrix(adj)
    adj_ = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
    # return sparse_to_tuple(adj_normalized)
    return sparse_mx_to_torch_sparse_tensor(adj_normalized)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [6]:
graph = pickle.load(open("pickles_data/graph_directed_networkx.pkl","rb"))
refs = pickle.load(open("pickles_data/graph.pkl","rb"))
id_to_name = pickle.load(open("pickles_data/arnetids_to_papername.pkl","rb"))
name_to_id = pickle.load(open("pickles_data/papername_to_arnetids.pkl","rb"))
tags = pickle.load(open("pickles_data/baseline_tags.pkl","rb"))
paper_arnet_id = pickle.load(open("pickles_data/paper_id_arnet_id.pickle","rb"))
dataset = []
count = 0
total_base = 0
extra_edges =  []
for key in tags :
    if(key in paper_arnet_id) :
        papers = tags[key]
        for paper in papers :
            name = paper['paper_name']
            if(name in name_to_id) :
                data = {}
                data['paper_name'] = name
                data['arnet_id'] = name_to_id[name]
                data['acl_paper_id'] = key
                data['acl_paper_arnetid'] = paper_arnet_id[key]
                data['tag'] = paper['tag']
                if(graph.has_edge(data['acl_paper_arnetid'],data['arnet_id'])==False) :
                    graph.add_edge(data['acl_paper_arnetid'],data['arnet_id'])
                    extra_edges.append((data['acl_paper_arnetid'],data['arnet_id']))
                    count+=1
                dataset.append(data)

print(len(dataset))
trainset = dataset[:int(0.8*(len(dataset)))]
testset = dataset[int(0.8*len(dataset)):]
train_base = 0
test_base = 0
for data in trainset :
    if(data['tag']==1) :
        train_base+=1
        
for data in testset :
    if(data['tag']==1) :
        test_base+=1
        
pickle.dump(testset,open("pickles_data/testset_for_graph.pkl","wb"))
pickle.dump(trainset,open("pickles_data/trainset_for_graph.pkl","wb"))
print(count)
print(total_base)
print(train_base)
print(test_base)
print(graph.number_of_edges())

29574
20467
0
2278
538
128567


In [7]:
def load_data(testset, graph) :  
    not_found = []
    for data in testset :
        arnet_acl = data['acl_paper_arnetid']
        arnet_ref = data['arnet_id']
        if(graph.has_edge(arnet_acl,arnet_ref)) :
            graph.remove_edge(arnet_acl, arnet_ref)
        else :
            not_found.append((arnet_acl, arnet_ref))
        
    adj = nx.adjacency_matrix(graph)
    
    nodes_order = graph.nodes()
    features = np.identity(len(nodes_order))
    
    return adj, features, nodes_order, not_found

In [8]:
adj, features, nodes_order, not_found = load_data(testset, graph)

In [9]:
print(len(not_found))

21


In [10]:
print(not_found)

[('53e9b321b7602d9703df0f46', '53e9b2b1b7602d9703d589f4'), ('53e9b321b7602d9703df0f46', '53e9a52bb7602d9702e4aecc'), ('5736974d6e3b12023e638925', '53e9a650b7602d9702f7ee22'), ('5736973a6e3b12023e6297c8', '53e9ab6fb7602d970350dc5d'), ('53e9a6f5b7602d9703028af8', '53e9b500b7602d970402eefa'), ('53e9a5c5b7602d9702eee617', '53e9b954b7602d970453d4d1'), ('5550446545ce0a409eb4d534', '53e9b108b7602d9703b85b88'), ('573696da6e3b12023e5d97c9', '53e99c6fb7602d9702522bf8'), ('5736974d6e3b12023e638699', '53e99822b7602d9702043ef5'), ('53e9a113b7602d9702a0150a', '53e9b661b7602d97041c0971'), ('53e9b755b7602d97042f56f6', '53e9b661b7602d97041c0971'), ('53e9b55db7602d9704097639', '53e9aeeab7602d97039146cc'), ('53e9b002b7602d9703a5b75e', '53e9b8a1b7602d970447a7a2'), ('53e9a073b7602d9702959b2a', '53e9a15bb7602d9702a507d8'), ('5550437f45ce0a409eb47d44', '53e9bbe5b7602d970483c59d'), ('53e9b388b7602d9703e6c8ce', '53e9b808b7602d97043bd219'), ('53e9b60db7602d97041629c3', '53e9a524b7602d9702e45758'), ('53e9a819b76

In [11]:
from torch import optim
import time

import numpy as np
import scipy.sparse as sp

adj_train = adj
n_nodes, feat_dim = features.shape
adj_norm = preprocess_graph(adj)
adj_label = adj_train + sp.eye(adj_train.shape[0])
# adj_label = sparse_to_tuple(adj_label)
adj_label = torch.FloatTensor(adj_label.toarray())

pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)
pos_weight = torch.FloatTensor(np.array(pos_weight)).cuda()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = GCNModelAE(feat_dim, 64, 32 , 0.1).to(device)
optimizer = optim.Adam(model.parameters(), 0.1)
features = torch.FloatTensor(features)
hidden_emb = None
for epoch in range(10000):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    features = features.cuda()
    adj_norm = adj_norm.cuda()
    recovered, embeddings = model(features, adj_norm)
    adj_label = adj_label.cuda()
    loss = loss_function(preds=recovered, labels=adj_label, norm=norm, pos_weight=pos_weight)
    loss.backward()
    cur_loss = loss.item()
    optimizer.step()

    hidden_emb = embeddings.cpu().data.numpy()

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cur_loss), "time=", "{:.5f}".format(time.time() - t))

print("Optimization Finished!")

Epoch: 0001 train_loss= 0.73421 time= 0.60490
Epoch: 0002 train_loss= 5.73786 time= 0.11931
Epoch: 0003 train_loss= 0.73849 time= 0.11653
Epoch: 0004 train_loss= 0.73398 time= 0.11566
Epoch: 0005 train_loss= 0.73392 time= 0.11564
Epoch: 0006 train_loss= 0.73415 time= 0.11552
Epoch: 0007 train_loss= 0.73458 time= 0.11663
Epoch: 0008 train_loss= 0.73457 time= 0.11671
Epoch: 0009 train_loss= 0.73425 time= 0.11651
Epoch: 0010 train_loss= 0.73393 time= 0.11568
Epoch: 0011 train_loss= 0.73377 time= 0.11570
Epoch: 0012 train_loss= 0.73360 time= 0.11693
Epoch: 0013 train_loss= 0.73360 time= 0.11624
Epoch: 0014 train_loss= 0.73325 time= 0.11555
Epoch: 0015 train_loss= 0.73314 time= 0.11653
Epoch: 0016 train_loss= 0.73287 time= 0.11666
Epoch: 0017 train_loss= 0.73230 time= 0.11656
Epoch: 0018 train_loss= 0.73230 time= 0.11576
Epoch: 0019 train_loss= 0.73150 time= 0.11632
Epoch: 0020 train_loss= 0.73130 time= 0.11605
Epoch: 0021 train_loss= 0.73054 time= 0.11677
Epoch: 0022 train_loss= 0.73045 ti

Epoch: 0181 train_loss= 0.62664 time= 0.11656
Epoch: 0182 train_loss= 0.62629 time= 0.11596
Epoch: 0183 train_loss= 0.62536 time= 0.11640
Epoch: 0184 train_loss= 0.62420 time= 0.11683
Epoch: 0185 train_loss= 0.62474 time= 0.11646
Epoch: 0186 train_loss= 0.62391 time= 0.11657
Epoch: 0187 train_loss= 0.62385 time= 0.11705
Epoch: 0188 train_loss= 0.62283 time= 0.11698
Epoch: 0189 train_loss= 0.62233 time= 0.11564
Epoch: 0190 train_loss= 0.62219 time= 0.11661
Epoch: 0191 train_loss= 0.62237 time= 0.11633
Epoch: 0192 train_loss= 0.62312 time= 0.11689
Epoch: 0193 train_loss= 0.62127 time= 0.11685
Epoch: 0194 train_loss= 0.62257 time= 0.11747
Epoch: 0195 train_loss= 0.62126 time= 0.11661
Epoch: 0196 train_loss= 0.62112 time= 0.11715
Epoch: 0197 train_loss= 0.61929 time= 0.11634
Epoch: 0198 train_loss= 0.62054 time= 0.11669
Epoch: 0199 train_loss= 0.61904 time= 0.11723
Epoch: 0200 train_loss= 0.62074 time= 0.11667
Epoch: 0201 train_loss= 0.61924 time= 0.11621
Epoch: 0202 train_loss= 0.62052 ti

Epoch: 0361 train_loss= 0.59366 time= 0.11644
Epoch: 0362 train_loss= 0.59190 time= 0.11757
Epoch: 0363 train_loss= 0.59413 time= 0.11702
Epoch: 0364 train_loss= 0.59509 time= 0.11605
Epoch: 0365 train_loss= 0.59285 time= 0.11668
Epoch: 0366 train_loss= 0.59288 time= 0.11665
Epoch: 0367 train_loss= 0.59390 time= 0.11785
Epoch: 0368 train_loss= 0.59288 time= 0.11627
Epoch: 0369 train_loss= 0.59303 time= 0.11581
Epoch: 0370 train_loss= 0.59192 time= 0.11682
Epoch: 0371 train_loss= 0.59385 time= 0.11680
Epoch: 0372 train_loss= 0.59342 time= 0.11660
Epoch: 0373 train_loss= 0.59107 time= 0.11639
Epoch: 0374 train_loss= 0.59196 time= 0.11617
Epoch: 0375 train_loss= 0.59109 time= 0.11667
Epoch: 0376 train_loss= 0.59164 time= 0.11651
Epoch: 0377 train_loss= 0.59137 time= 0.11661
Epoch: 0378 train_loss= 0.59088 time= 0.11666
Epoch: 0379 train_loss= 0.59027 time= 0.11683
Epoch: 0380 train_loss= 0.59217 time= 0.11679
Epoch: 0381 train_loss= 0.59077 time= 0.11640
Epoch: 0382 train_loss= 0.58984 ti

Epoch: 0541 train_loss= 0.57963 time= 0.11639
Epoch: 0542 train_loss= 0.58072 time= 0.11689
Epoch: 0543 train_loss= 0.58141 time= 0.11652
Epoch: 0544 train_loss= 0.58052 time= 0.11675
Epoch: 0545 train_loss= 0.57856 time= 0.11647
Epoch: 0546 train_loss= 0.57986 time= 0.11652
Epoch: 0547 train_loss= 0.57869 time= 0.11633
Epoch: 0548 train_loss= 0.57997 time= 0.11666
Epoch: 0549 train_loss= 0.57978 time= 0.11566
Epoch: 0550 train_loss= 0.57989 time= 0.11681
Epoch: 0551 train_loss= 0.58004 time= 0.11712
Epoch: 0552 train_loss= 0.58094 time= 0.11654
Epoch: 0553 train_loss= 0.57969 time= 0.11605
Epoch: 0554 train_loss= 0.57926 time= 0.11660
Epoch: 0555 train_loss= 0.58029 time= 0.11640
Epoch: 0556 train_loss= 0.58142 time= 0.11593
Epoch: 0557 train_loss= 0.57932 time= 0.11652
Epoch: 0558 train_loss= 0.57993 time= 0.11640
Epoch: 0559 train_loss= 0.57954 time= 0.11620
Epoch: 0560 train_loss= 0.57973 time= 0.11664
Epoch: 0561 train_loss= 0.58019 time= 0.11619
Epoch: 0562 train_loss= 0.57835 ti

Epoch: 0721 train_loss= 0.57682 time= 0.11643
Epoch: 0722 train_loss= 0.57570 time= 0.11641
Epoch: 0723 train_loss= 0.57628 time= 0.11638
Epoch: 0724 train_loss= 0.57422 time= 0.11652
Epoch: 0725 train_loss= 0.57730 time= 0.11648
Epoch: 0726 train_loss= 0.57584 time= 0.11585
Epoch: 0727 train_loss= 0.57585 time= 0.11644
Epoch: 0728 train_loss= 0.57670 time= 0.11660
Epoch: 0729 train_loss= 0.57659 time= 0.11645
Epoch: 0730 train_loss= 0.57495 time= 0.11653
Epoch: 0731 train_loss= 0.57602 time= 0.11636
Epoch: 0732 train_loss= 0.57659 time= 0.11670
Epoch: 0733 train_loss= 0.57573 time= 0.11843
Epoch: 0734 train_loss= 0.57518 time= 0.11601
Epoch: 0735 train_loss= 0.57549 time= 0.11619
Epoch: 0736 train_loss= 0.57646 time= 0.11658
Epoch: 0737 train_loss= 0.57831 time= 0.11559
Epoch: 0738 train_loss= 0.57493 time= 0.11724
Epoch: 0739 train_loss= 0.57586 time= 0.11714
Epoch: 0740 train_loss= 0.57705 time= 0.11582
Epoch: 0741 train_loss= 0.57661 time= 0.11676
Epoch: 0742 train_loss= 0.57506 ti

...

Epoch: 9181 train_loss= 0.53603 time= 0.11633
Epoch: 9182 train_loss= 0.53374 time= 0.11621
Epoch: 9183 train_loss= 0.53391 time= 0.11631
Epoch: 9184 train_loss= 0.53441 time= 0.11593
Epoch: 9185 train_loss= 0.53425 time= 0.11651
Epoch: 9186 train_loss= 0.53386 time= 0.11647
Epoch: 9187 train_loss= 0.53606 time= 0.11671
Epoch: 9188 train_loss= 0.53438 time= 0.11735
Epoch: 9189 train_loss= 0.53303 time= 0.11552
Epoch: 9190 train_loss= 0.53321 time= 0.11653
Epoch: 9191 train_loss= 0.53368 time= 0.11636
Epoch: 9192 train_loss= 0.53316 time= 0.11756
Epoch: 9193 train_loss= 0.53632 time= 0.11729
Epoch: 9194 train_loss= 0.53470 time= 0.11625
Epoch: 9195 train_loss= 0.53385 time= 0.11644
Epoch: 9196 train_loss= 0.53482 time= 0.11649
Epoch: 9197 train_loss= 0.53556 time= 0.11635
Epoch: 9198 train_loss= 0.53483 time= 0.11683
Epoch: 9199 train_loss= 0.53453 time= 0.11603
Epoch: 9200 train_loss= 0.53374 time= 0.11643
Epoch: 9201 train_loss= 0.53419 time= 0.11644
Epoch: 9202 train_loss= 0.53434 ti

Epoch: 9361 train_loss= 0.53342 time= 0.11633
Epoch: 9362 train_loss= 0.53470 time= 0.11673
Epoch: 9363 train_loss= 0.53310 time= 0.11645
Epoch: 9364 train_loss= 0.53453 time= 0.11695
Epoch: 9365 train_loss= 0.53409 time= 0.11707
Epoch: 9366 train_loss= 0.53620 time= 0.11622
Epoch: 9367 train_loss= 0.53685 time= 0.11717
Epoch: 9368 train_loss= 0.53608 time= 0.11633
Epoch: 9369 train_loss= 0.53276 time= 0.11717
Epoch: 9370 train_loss= 0.53408 time= 0.11710
Epoch: 9371 train_loss= 0.53504 time= 0.11635
Epoch: 9372 train_loss= 0.53574 time= 0.11680
Epoch: 9373 train_loss= 0.53609 time= 0.11652
Epoch: 9374 train_loss= 0.53480 time= 0.11631
Epoch: 9375 train_loss= 0.53506 time= 0.11584
Epoch: 9376 train_loss= 0.53499 time= 0.11756
Epoch: 9377 train_loss= 0.53460 time= 0.11699
Epoch: 9378 train_loss= 0.53374 time= 0.11716
Epoch: 9379 train_loss= 0.53266 time= 0.11571
Epoch: 9380 train_loss= 0.53324 time= 0.11788
Epoch: 9381 train_loss= 0.53359 time= 0.11738
Epoch: 9382 train_loss= 0.53596 ti

Epoch: 9541 train_loss= 0.53615 time= 0.11768
Epoch: 9542 train_loss= 0.53559 time= 0.11680
Epoch: 9543 train_loss= 0.53429 time= 0.11670
Epoch: 9544 train_loss= 0.53433 time= 0.11612
Epoch: 9545 train_loss= 0.53473 time= 0.11900
Epoch: 9546 train_loss= 0.53420 time= 0.11688
Epoch: 9547 train_loss= 0.53357 time= 0.11557
Epoch: 9548 train_loss= 0.53537 time= 0.11656
Epoch: 9549 train_loss= 0.53400 time= 0.11707
Epoch: 9550 train_loss= 0.53392 time= 0.11681
Epoch: 9551 train_loss= 0.53394 time= 0.11551
Epoch: 9552 train_loss= 0.53511 time= 0.11639
Epoch: 9553 train_loss= 0.53627 time= 0.11645
Epoch: 9554 train_loss= 0.53618 time= 0.11610
Epoch: 9555 train_loss= 0.53572 time= 0.11645
Epoch: 9556 train_loss= 0.53268 time= 0.11532
Epoch: 9557 train_loss= 0.53581 time= 0.11633
Epoch: 9558 train_loss= 0.53371 time= 0.11580
Epoch: 9559 train_loss= 0.53595 time= 0.11625
Epoch: 9560 train_loss= 0.53542 time= 0.11664
Epoch: 9561 train_loss= 0.53488 time= 0.11637
Epoch: 9562 train_loss= 0.53361 ti

Epoch: 9721 train_loss= 0.53965 time= 0.11705
Epoch: 9722 train_loss= 0.53626 time= 0.11607
Epoch: 9723 train_loss= 0.53754 time= 0.11642
Epoch: 9724 train_loss= 0.53812 time= 0.11715
Epoch: 9725 train_loss= 0.53508 time= 0.11656
Epoch: 9726 train_loss= 0.53372 time= 0.11691
Epoch: 9727 train_loss= 0.53564 time= 0.11656
Epoch: 9728 train_loss= 0.53632 time= 0.11667
Epoch: 9729 train_loss= 0.53683 time= 0.11655
Epoch: 9730 train_loss= 0.53421 time= 0.11542
Epoch: 9731 train_loss= 0.53548 time= 0.11709
Epoch: 9732 train_loss= 0.53325 time= 0.11674
Epoch: 9733 train_loss= 0.53551 time= 0.11566
Epoch: 9734 train_loss= 0.53675 time= 0.11650
Epoch: 9735 train_loss= 0.53635 time= 0.11676
Epoch: 9736 train_loss= 0.53461 time= 0.11655
Epoch: 9737 train_loss= 0.53515 time= 0.11678
Epoch: 9738 train_loss= 0.53338 time= 0.11676
Epoch: 9739 train_loss= 0.53512 time= 0.11678
Epoch: 9740 train_loss= 0.53607 time= 0.11659
Epoch: 9741 train_loss= 0.53517 time= 0.11709
Epoch: 9742 train_loss= 0.53442 ti

Epoch: 9901 train_loss= 0.53520 time= 0.11520
Epoch: 9902 train_loss= 0.53364 time= 0.11541
Epoch: 9903 train_loss= 0.53411 time= 0.11516
Epoch: 9904 train_loss= 0.53575 time= 0.11543
Epoch: 9905 train_loss= 0.53487 time= 0.11547
Epoch: 9906 train_loss= 0.53489 time= 0.11609
Epoch: 9907 train_loss= 0.53583 time= 0.11541
Epoch: 9908 train_loss= 0.53389 time= 0.11525
Epoch: 9909 train_loss= 0.53558 time= 0.11511
Epoch: 9910 train_loss= 0.53371 time= 0.11518
Epoch: 9911 train_loss= 0.53326 time= 0.11580
Epoch: 9912 train_loss= 0.53530 time= 0.11547
Epoch: 9913 train_loss= 0.53332 time= 0.11512
Epoch: 9914 train_loss= 0.53560 time= 0.11628
Epoch: 9915 train_loss= 0.53445 time= 0.11550
Epoch: 9916 train_loss= 0.53493 time= 0.11522
Epoch: 9917 train_loss= 0.53532 time= 0.11514
Epoch: 9918 train_loss= 0.53489 time= 0.11521
Epoch: 9919 train_loss= 0.53349 time= 0.11517
Epoch: 9920 train_loss= 0.53343 time= 0.11514
Epoch: 9921 train_loss= 0.53788 time= 0.11545
Epoch: 9922 train_loss= 0.53412 ti

In [12]:
hidden_emb.shape

(14594, 32)

In [13]:
adj.shape

(14594, 14594)

In [14]:
pickle.dump(hidden_emb, open("pickles_data/graph_directed_embedding_32.pkl","wb"))

In [15]:
pickle.dump(nodes_order, open("pickles_data/graph_directed_nodes_map.pkl","wb"))

In [16]:
type(nodes_order)

networkx.classes.reportviews.NodeView

In [17]:
mapping = {}
node_list = list(nodes_order)
for i in range(len(node_list)):
    mapping[node_list[i]] = hidden_emb[i]    

In [18]:
print(len(mapping.keys()))

14594


In [19]:
pickle.dump(mapping, open("pickles_data/graph_directed_embedding_mapping.pkl","wb"))